# 필요한 라이브러리 Import

In [1]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model
import random
import numpy as np
from tqdm import tqdm, trange, tqdm_notebook
import pandas as pd
from ast import literal_eval
from torch import nn
from transformers import BertModel, BertConfig
from torchcrf import CRF
from pytorch_transformers import AdamW, WarmupLinearSchedule
from gluonnlp.data import BERTSPTokenizer
import sentencepiece as spm
from seqeval.metrics import f1_score
from pathlib import Path
import json

In [2]:
class Config:
    def __init__(self, json_path):
        with open(json_path, mode='r') as io:
            params = json.loads(io.read())
        self.__dict__.update(params)

    def save(self, json_path):
        with open(json_path, mode='w') as io:
            json.dump(self.__dict__, io, indent=4)

    def update(self, json_path):
        with open(json_path, mode='r') as io:
            params = json.loads(io.read())
        self.__dict__.update(params)

    @property
    def dict(self):
        return self.__dict__

# KoBERT + CRF

In [3]:
#클래스 참고: KoBERT와 CRF로 만든 한국어 개체명 인식 https://github.com/eagle705/pytorch-bert-crf-ner

bert_config = {'attention_probs_dropout_prob': 0.1,
                 'hidden_act': 'gelu',
                 'hidden_dropout_prob': 0.1,
                 'hidden_size': 768,
                 'initializer_range': 0.02,
                 'intermediate_size': 3072,
                 'max_position_embeddings': 512,
                 'num_attention_heads': 12,
                 'num_hidden_layers': 12,
                 'type_vocab_size': 2,
                 'vocab_size': 11575,
                 'padding': True,
                 'pair': False
                 }
                 
class KobertCRF(nn.Module):
    """ KoBERT with CRF """
    def __init__(self, config, num_classes, vocab=None) -> None:
        super(KobertCRF, self).__init__()

        self.bert = BertModel(config=BertConfig.from_dict(bert_config))
        self.vocab = vocab

        self.dropout = nn.Dropout(config.dropout)
        self.position_wise_ff = nn.Linear(config.hidden_size, num_classes)
        self.crf = CRF(num_tags=num_classes, batch_first=True)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
        
    def forward(self, input_ids, valid_length, token_type_ids=None, tags=None):
        attention_mask = self.gen_attention_mask(input_ids, valid_length)

        # outputs: (last_encoder_layer, pooled_output, attention_weight)
        outputs = self.bert(input_ids=input_ids,
                            token_type_ids=token_type_ids,
                            attention_mask=attention_mask.float().to(input_ids.device))
        last_encoder_layer = outputs[0]
        last_encoder_layer = self.dropout(last_encoder_layer)
        emissions = self.position_wise_ff(last_encoder_layer)

        if tags is not None:
            log_likelihood, sequence_of_tags = self.crf(emissions, tags), self.crf.decode(emissions)
            return log_likelihood, sequence_of_tags
        else:
            sequence_of_tags = self.crf.decode(emissions)
            return sequence_of_tags

# Tokenizer & Vocab 생성


In [4]:
#Tokenizer model 생성

corpus = "vocab.txt" # KoCharELECTRA (https://github.com/monologg/KoCharELECTRA) vocab.txt 파일 사용 (음절 vocab 파일)
prefix = "vocab"
vocab_size = 11568 # vocab.txt 크기

spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=10" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

In [5]:
vocab = nlp.vocab.BERTVocab.from_sentencepiece('vocab.model', padding_token='[PAD]')
tokenizer = "vocab.model" #생성한 vocab.model 이용
tok = BERTSPTokenizer(tokenizer, vocab, lower=False) # BERTSPTokenizer 생성|

## 데이터 셋 불러오기

In [6]:
import re
p = re.compile('[一-龥]') #한자를 포함한 문장 제거하기 위한 정규식

train_list_csv = pd.read_csv('Data/pet_train_V3.csv', converters={"tag": literal_eval})
test_list_csv = pd.read_csv('Data/pet_test_V3.csv', converters={"tag": literal_eval})

tr_tag = train_list_csv['tag']
tr_sent = train_list_csv['sent']
ts_tag = test_list_csv['tag']
ts_sent = test_list_csv['sent']

for i in range(len(train_list_csv['tag'])):
    if p.search(tr_sent[i]): #한자 문장 제거
        del tr_sent[i]
        del tr_tag[i]
    elif len(tr_tag[i]) != len(tr_sent[i])+3: #데이터 셋 준비 과정 중 길이 다르면 해당 인댁스 출력 +3 해주는 이유는 버트에 넣기 전 붙을 태그 등 데이터 떄문
        print(i)
    elif len(tr_tag[i]) > 512 or len(tr_sent[i])+3 > 512: # 문장의 길이가 512가 넘는 문장 제거
        del tr_sent[i]
        del tr_tag[i]

for i in range(len(test_list_csv['tag'])):
    if p.search(ts_sent[i]): #한자 문장 제거
        del ts_sent[i]
        del ts_tag[i]
    elif 'Ⅳ' in ts_sent[i]: #로마자 제거
        del ts_sent[i]
        del ts_tag[i]
    elif len(ts_tag[i]) != len(ts_sent[i])+3: #데이터 셋 준비 과정 중 길이 다르면 해당 인댁스 출력 +3 해주는 이유는 버트에 넣기 전 붙을 태그 등 데이터 떄문
        print(i)
    elif len(ts_tag[i]) > 512 or len(ts_sent[i])+3 > 512: # 문장의 길이가 512가 넘는 문장 제거
        del ts_sent[i]
        del ts_tag[i]
    

## 버트 데이터 셋 클래스 생성

In [7]:
# 코드 참고: 버트를 이용한 네이버 댓글 분류기 https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

class BERTDataset(Dataset):
    def __init__(self, sent, tag, bert_tokenizer, max_len,
                pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_len, pad=pad, pair=pair)
        self.sentences = [transform([i]) for i in sent] #문장
        self.labels = [np.pad(np.int32(i), (0,max_len-len(i)), 'constant', constant_values=0) for i in tag] #질병 태그 및 질병 태그 max_len 패딩

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

## 사용자 입력 문장을 테스하기 위한 데이터 셋 클래스 생성

In [7]:
class BERTTESTsent(Dataset):
    def __init__(self, sent, bert_tokenizer, max_len,
                pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_len, pad=pad, pair=pair)
        self.sentences = [transform([i]) for i in sent] #문장

    def __getitem__(self, i):
        return (self.sentences[i])

    def __len__(self):
        return (len(self.sentences))

## 데이터 셋 나누기

In [8]:
data_train = BERTDataset(tr_sent,tr_tag, tok, 512, True, False)
data_test = BERTDataset(ts_sent,ts_tag, tok, 512, True, False)

In [9]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=32, num_workers = 5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=1, num_workers = 5, shuffle=True)

In [ ]:
data_train[0]

## 데이터 확인

In [ ]:
def count_len(list):
    count = 0
    for i in range(len(list)):
        if list[i] == 0:
            break
        else:
            count += 1
    return count

In [ ]:
data_tr_count = 0
data_ts_count = 0
for i in range(len(data_train)):
    
    if count_len(data_train[i][0]) != count_len(data_train[i][3]):
        data_tr_count += 1
        print(i)

for i in tqdm_notebook(range(len(data_test))):

    if count_len(data_test[i][0]) != count_len(data_test[i][3]):
        data_ts_count += 1
        print(i)
print(data_tr_count, data_ts_count)

# 모델 선언 (초기 학습을 위한)

In [10]:
device = torch.device('cuda')
model_config = Config('config.json')
model = KobertCRF(config=model_config, num_classes=7)
model.to(device)
model.train()

KobertCRF(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(11575, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

# 학습된 모델 불러오기 (파인 튜닝을 위한)

In [ ]:
model = torch.load('KoBERT_DIS_NER_V3/Human_DIS_V3(epoch2, step1100).pth')
model_config = Config('config.json')

# 모델 환경 설정

In [11]:
def set_seed(seed=100):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    n_gpu = torch.cuda.device_count()
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

In [12]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(
        nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

t_total = len(
    train_dataloader) // model_config.gradient_accumulation_steps * model_config.epochs
optimizer = AdamW(optimizer_grouped_parameters,
                    lr=model_config.learning_rate, eps=model_config.adam_epsilon)
scheduler = WarmupLinearSchedule(
    optimizer, warmup_steps=model_config.warmup_steps, t_total=t_total)

model_dir = 'DIS_NER/'

global_step = 0
tr_loss, logging_loss = 0.0, 0.0
model.zero_grad()
set_seed()

# 모델 학습

In [ ]:
import os

In [16]:
train_iterator = trange(int(model_config.epochs), desc="Epoch")
device = torch.device('cuda')

for _epoch, _ in enumerate(train_iterator):
    epoch = _epoch
    model.train()
    for step, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        log_likelihood, sequence_of_tags = model(token_ids,valid_length, segment_ids, label)
        
        # loss: negative log-likelihood
        loss = -1 * log_likelihood
            
        loss.backward()
        torch.nn.utils.clip_grad_norm_(
            model.parameters(), model_config.max_grad_norm)
        tr_loss += loss.item()

        if (step + 1) % model_config.gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            model.zero_grad()
            global_step += 1

            with torch.no_grad():
                sequence_of_tags = torch.tensor(
                    sequence_of_tags).to(device)
                mb_acc = (sequence_of_tags == label).float()[label != 0].mean()

            tr_acc = mb_acc.item()
            tr_loss_avg = tr_loss / global_step
            tr_summary = {'loss': tr_loss_avg, 'acc': tr_acc}
            state = {'global_step': global_step + 1,
                             'model_state_dict': model.state_dict(),
                             'opt_state_dict': optimizer.state_dict()}
            if step % 100 == 0:
                print('epoch : {}, global_step : {}, tr_loss: {:.3f}, tr_acc: {:.2%}'.format(
                        epoch + 1, global_step, tr_summary['loss'], tr_summary['acc']))  
                torch.save(model, 'KoBERT_DIS_NER_V3/Human_DIS_V4(epoch'+str(epoch+1)+", step"+str(step)+').pth')
    print("epoch finish")

writer.close()

Epoch:   0%|          | 0/150 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


torch.Size([32, 512])


/RNTier/home/halee/.local/lib/python3.6/site-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


epoch : 1, global_step : 1, tr_loss: 31921.584, tr_acc: 7.61%
torch.Size([32, 512])


Epoch:   0%|          | 0/150 [00:05<?, ?it/s]

KeyboardInterrupt: 

# 모델 평가

In [ ]:
def index_to_ner(sequences, val_len):

    idx_to_ner = {
        1 : "[CLS]", 
        2 : "[SEP]", 
        0 : "O", #[PAD] to 'O'
        3 : "[MASK]",
        4 : "O",
        5 : "B-DIS",
        6 : "I-DIS"
    }
    result = []
    for i in range(val_len):
        result.append(idx_to_ner[sequences[i]])
    return result

In [ ]:
device = torch.device('cuda')
model_config = Config('config.json')
model = torch.load('Human_Pet_DIS_V3(epoch3, step300).pth')
model.to(device)
model.eval()
vocab_file = "vocab.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)
eval_step = 0
sum_f1 = 0
predic_list = []
true_list = []
dis_list2 = []
for step, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)

    log_likelihood, sequence_of_tags = model(token_ids,valid_length, segment_ids, label)
    # loss: negative log-likelihood
    eval_loss = -1 * log_likelihood
    
    temp_sent = ''
    
    for i in range(int(valid_length)):
        if sequence_of_tags[0][i] == 5 or sequence_of_tags[0][i] == 6:
            if sequence_of_tags[0][i] == 5 and len(temp_sent) > 0:
                dis_list2.append(temp_sent)
                temp_sent = vocab.IdToPiece(int(token_ids[0][i]))
            else:
                temp_sent += vocab.IdToPiece(int(token_ids[0][i]))
        elif len(temp_sent) > 0:
            dis_list2.append(temp_sent)
            temp_sent = ''
            
    if (eval_step + 1) % model_config.gradient_accumulation_steps == 0:
        eval_step += 1
        with torch.no_grad():
            sequence_of_tags = torch.tensor(
                sequence_of_tags).to(device)
            eval_acc = (sequence_of_tags == label).float()[label != 0].mean()
        eval_loss_avg = eval_loss / eval_step
        eval_summary = {'loss': eval_loss_avg, 'acc': eval_acc}

        predic_list.append(index_to_ner(sequence_of_tags[0].cpu().numpy(), valid_length[0].cpu().numpy()))
        true_list.append(index_to_ner(label[0].cpu().numpy(), valid_length[0].cpu().numpy()))
        
        if eval_step % 10 == 0:
            print('eval_step : {}, evl_loss: {:.3f}, evl_acc: {:.2f}'.format(
                eval_step, eval_summary['loss'], eval_summary['acc']))

## F1 스코어 평가

In [ ]:
f1_score(true_list, predic_list)

In [ ]:
dis_list2 = list(set(dis_list2))
f = open("humanModel_pet_test_dis_list_V3.txt", 'w')

for line in dis_list2:
    f.write(line+'\n')
f.close()

# 사용자 입력 문장 모델 평가

In [10]:
vocab = nlp.vocab.BERTVocab.from_sentencepiece('vocab.model', padding_token='[PAD]')
tokenizer = "vocab.model" #생성한 vocab.model 이용
tok = BERTSPTokenizer(tokenizer, vocab, lower=False) # BERTSPTokenizer 생성

In [39]:
model_config = Config('config.json')

# model
model = torch.load('KoBERT_DIS_NER_V3/Human_Pet_DIS_V3(epoch3, step300).pth')
model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
vocab_file = "vocab.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)
input_text = input('input> ')
print(input_text)
text_sent = []
text_sent.append(input_text)
sent_test = BERTTESTsent(text_sent, tok, 512, True, False)
sent_dataloader = torch.utils.data.DataLoader(sent_test, batch_size=1, num_workers = 5)
temp_sent = ''
dis_list = []

for _, (token_ids, valid_length, segment_ids) in enumerate(tqdm_notebook(sent_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids =  segment_ids.long().to(device)
    valid_length =  valid_length
    sequence_of_tags = model(token_ids, valid_length, segment_ids)
    
    for i in range(int(valid_length)):
        if sequence_of_tags[0][i] == 5 or sequence_of_tags[0][i] == 6:
            if sequence_of_tags[0][i] == 5 and len(temp_sent) > 0:
                dis_list.append(temp_sent)
                temp_sent = vocab.IdToPiece(int(token_ids[0][i]))
            else:
                temp_sent += vocab.IdToPiece(int(token_ids[0][i]))
        elif len(temp_sent) > 0:
            dis_list.append(temp_sent)
            temp_sent = ''
print(dis_list)

input> 병원에서 정확한 진단과 적절한 처치 받아보시길 권해드립니다.
병원에서 정확한 진단과 적절한 처치 받아보시길 권해드립니다.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



['▁진']


In [36]:
def ner_make(model2, input_text):
    
    # model
    model = model2
    model.eval()
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    vocab_file = "vocab.model"
    vocab = spm.SentencePieceProcessor()
    vocab.load(vocab_file)
    model.to(device)
    text_sent = []
    text_sent.append(input_text)
    sent_test = BERTTESTsent(text_sent, tok, 512, True, False)
    sent_dataloader = torch.utils.data.DataLoader(sent_test, batch_size=1, num_workers = 5)
    temp_sent = ''
    dis_list = []

    for _, (token_ids, valid_length, segment_ids) in enumerate(sent_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids =  segment_ids.long().to(device)
        valid_length =  valid_length
        sequence_of_tags = model(token_ids, valid_length, segment_ids)

        for i in range(int(valid_length)):
            if sequence_of_tags[0][i] == 5 or sequence_of_tags[0][i] == 6:
                if sequence_of_tags[0][i] == 5 and len(temp_sent) > 0:
                    dis_list.append(temp_sent)
                    temp_sent = vocab.IdToPiece(int(token_ids[0][i]))
                else:
                    temp_sent += vocab.IdToPiece(int(token_ids[0][i]))
            elif len(temp_sent) > 0:
                dis_list.append(temp_sent)
                temp_sent = ''
    return dis_list

In [ ]:
print(sequence_of_tags)

In [ ]:
import json
import tqdm

# anago model load
anago_pet_ner_model = torch.load('KoBERT_DIS_NER_V3/Human_Pet_DIS_V3(epoch3, step300).pth')
# QA Data load
with open("Data/Vet_Data_Total_Not_Null_Question.json", "r", encoding='utf-8') as pet_data_json:
    pet_data = json.load(pet_data_json)
json_file_resultList = []
cnt = 0
for i in tqdm_notebook(pet_data):
    title = i['title']
    question = i['question']
    answer = i['answer']
    url = i['url']
    answer_test = answer.split('.')
    if (answer_test[-1] == '') or answer_test[-1] == ' ':
        del (answer_test[-1])
    answer_test = [i + "." for i in answer_test]
    answer_ner = []  # 개체명 인식 결과 저장 리스트
    sentence_info = []  # 개체명 인식 결과 문장 리스트
    for sentence in answer_test:
        sentence_tmp = sentence
        sentence = sentence.replace(' ', '')
        a = ner_make(anago_pet_ner_model, sentence)
        
        if len(a) > 0:
            for tmp in a:
                answer_ner.append(tmp)
            sentence_info.append(sentence_tmp)
    # 결과 확인 및 개체명 생성 완료된 json file 생성
    # set -> list : 중복제거
    answer_ner = set(answer_ner)
    answer_ner = list(answer_ner)
    # set -> list : 중복제거
    sentence_info = set(sentence_info)
    sentence_info = list(sentence_info)
    result = {
        "title": title,
        "question": question,
        "answer": answer,
        "answer_ner": str(answer_ner),
        "sentence_info": str(sentence_info),
        "url": url
    }
    json_file_resultList.append(result)
    if cnt % 100 == 0:
        print(cnt)
    cnt+=1
# # json 파일 저장



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


0


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fd7a863eb70>>
Traceback (most recent call last):
  File "/RNTier/home/halee/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/RNTier/home/halee/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1174, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100


In [43]:
with open(
        'Data/Vet_Data_Total_Not_Null_Question_NER.json',
        'w', encoding="utf-8") as make_file:
    json.dump(json_file_resultList, make_file, ensure_ascii=False, indent="\t")